In [143]:
import nltk

# download the VADER lexicon
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [144]:
import requests
import json
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [145]:
# list of company names to search for
company_names = ['Tesla', 'Apple', 'Google', 'VMware', 'Opera', 'Renault', 'SpaceX', 'Microsoft', 'samsung', 'Amazon', 'Wipro']

In [146]:
# get the Tech news page url
TOI_tech_news_url = "https://timesofindia.indiatimes.com/tech"
tech_new_link = BeautifulSoup(requests.get(TOI_tech_news_url).content, 'html.parser').find('div', {'class': '_1VsHa'}).find('a')['href']

website = TOI_tech_news_url + tech_new_link

In [147]:
# function to crawl the news websites and extract the relevant information
def crawl_news():
    articles = []
    # Make a request to the website
    response = requests.get(website)

    # Define the list of company names to rank
    company_names = ['Tesla', 'Apple', 'Google', 'SpaceX', 'Microsoft', 'Amazon', 'Wipro']

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the section of the website that contains links to articles
    ul = soup.find('ul', {'class': "VdmSM"})
    li_lists  = ul.find_all('li')[1:-1]
    page_links = [li.find('a') for li in li_lists]

    
    # Loop through each article link on the website
    for page_link in page_links:
        # Make a request to the page that contains the links to articles
        techpage = requests.get("https://www.gadgetsnow.com/"+page_link['href'])
         # Parse the HTML content of the page that contains the links to articles using BeautifulSoup
        techpage_soup = BeautifulSoup(techpage.content, 'html.parser')
        # Find the div elements that contain links to articles
        div_lists = techpage_soup.find('div', {'class': "_3u858"}).find_all('div', {'class': "_3IvxE"})
        article_links = [div.find('a') for div in div_lists]
        # Loop through each article link
        for link in article_links:
          # Make a request to the article link
          article_responce = requests.get(link['href'])
          article_soup = BeautifulSoup(article_responce.content, 'html.parser')
          # Extract the article title and content
          article_title = article_soup.find('title').get_text()
          article_content = ''
          for span in article_soup.find('div',{'class': '_3Aytd'}).find_all('span'):
            article_content += span.get_text()
          # Loop through each company name to determine if it is mentioned in the article content
          for company in company_names:
                if company in article_content:
                    articles.append({'title': article_title, 'link': link['href'], 'content': article_content, 'company': company})
    return articles


In [148]:
# function to rank the articles based on relevance and sentiment towards the company
def rank_articles(articles):
    sid = SentimentIntensityAnalyzer()
    for article in articles:
        score = 0
        if article['company'] in article['title']:
            score += 3
        if article['company'] in article['content']:
            sentiment = sid.polarity_scores(article['content'])
            score += sentiment['compound']
        article['score'] = score
    ranked_articles = sorted(articles, key=lambda k: k['score'], reverse=True)
    for article in ranked_articles:
        if article['score'] >= 0:
            article['sentiment'] = 'positive'
        else:
            article['sentiment'] = 'negative'
    return ranked_articles

In [149]:
# call the functions and print the results
articles = crawl_news()
ranked_articles = rank_articles(articles)

In [150]:
for article in ranked_articles:
    x = {'Company': str(article['company']), 'Sentiment Analysis': str(article['sentiment']), 'Sentiment Score': str(article['score']), 'Title': str(article['title']), 'Article Link': str(article['link'])}
    y = json.dumps(x)
    print(y)

{"Company": "Google", "Sentiment Analysis": "positive", "Sentiment Score": "3.9886", "Title": "NetCore partners with Google Cloud for email and SMS API", "Article Link": "https://www.gadgetsnow.com/tech-news/netcore-partners-with-google-cloud-for-email-and-sms-api/articleshow/99792980.cms"}
{"Company": "Google", "Sentiment Analysis": "positive", "Sentiment Score": "3.9884", "Title": "Google\u2019s multi-billion dollar campus is on \u2018hold\u2019, here\u2019s why", "Article Link": "https://www.gadgetsnow.com/tech-news/googles-multi-billion-dollar-campus-is-on-hold-heres-why/articleshow/99702307.cms"}
{"Company": "Tesla", "Sentiment Analysis": "positive", "Sentiment Score": "3.9882999999999997", "Title": "tesla: Tesla encourages owners to give up 'unlimited free Supercharging for life'", "Article Link": "https://www.gadgetsnow.com/tech-news/tesla-encourages-owners-to-give-up-unlimited-free-supercharging-for-life/articleshow/99737150.cms"}
{"Company": "Google", "Sentiment Analysis": "po